In [14]:
import pandas  as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# Unbalanced Prediction

In [ ]:
data = pd.read_csv('src/predictive_maintenance.csv')
print(data.info)

In [16]:
new_labels = ['uid', 'prod_id', 'prod_type', 'air_temp', 'process_temp', 'rot_speed', 'torque', 'tool_wear', 'target', 'failure_type']

data.columns = new_labels
X = data[['torque','process_temp','tool_wear', "air_temp", "rot_speed"]]
y = data['target']
print(y.value_counts())

target
0    9661
1     339
Name: count, dtype: int64


In [18]:
from sklearn.model_selection import train_test_split

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Visualizando o shape dos dados
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(7000, 5) (3000, 5) (7000,) (3000,)


In [19]:
# Treinando modelo sem balanceamento de classes
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train.ravel())

predictions = lr.predict(X_test)

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2907
           1       0.69      0.27      0.39        93

    accuracy                           0.97      3000
   macro avg       0.84      0.63      0.69      3000
weighted avg       0.97      0.97      0.97      3000



# NEAR MISS $\to$ Redução de Dimensionalidade


In [20]:
# Antes da redução de dimensionalidade

print(f"Antes da redução de dimensionalidade -> Falha: {sum(y_train ==1)}")
print(f"Antes da redução de dimensionalidade -> Não falha: {sum(y_train == 0)}")

# Aplicando o NearMiss
from imblearn.under_sampling import NearMiss

nr = NearMiss()

X_train_miss, y_train_miss = nr.fit_resample(X_train, y_train.ravel())

print(f"Depois da redução de dimensionalidade -> Falha: {sum(y_train_miss ==1)}")

print(f"Depois da redução de dimensionalidade -> Não falha: {sum(y_train_miss == 0)}")

Antes da redução de dimensionalidade -> Falha: 246
Antes da redução de dimensionalidade -> Não falha: 6754
Depois da redução de dimensionalidade -> Falha: 246
Depois da redução de dimensionalidade -> Não falha: 246


In [21]:
# Verificando resultados do NearMiss

lr2 = LogisticRegression()
lr2.fit(X_train_miss, y_train_miss.ravel())

predictions = lr2.predict(X_test)

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.99      0.65      0.79      2907
           1       0.06      0.72      0.11        93

    accuracy                           0.66      3000
   macro avg       0.52      0.69      0.45      3000
weighted avg       0.96      0.66      0.77      3000



# SMOTE

In [22]:
# Aplicando SMOTE para balancear os dados
# Criando superamostragem

print(f"Antes da superamostragem: Não Falha: {sum(y_train == 0)}")
print(f"Antes da superamostragem: Falha: {sum(y_train == 1)}\n")

# Aplicando Superamostragem

from imblearn.over_sampling import SMOTE

sm= SMOTE(random_state=2)

X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

print(f"Depois da superamostragem: Não Falha: {sum(y_train_sm == 0)}")	
print(f"Depois da superamostragem: Falha: {sum(y_train_sm == 1)}\n")


Antes da superamostragem: Não Falha: 6754
Antes da superamostragem: Falha: 246

Depois da superamostragem: Não Falha: 6754
Depois da superamostragem: Falha: 6754



In [23]:
# Criando regressao e verificando desempenho com SMOTE

lr1 =  LogisticRegression()

lr1.fit(X_train_sm, y_train_sm)

predictions = lr1.predict(X_test)

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.99      0.83      0.91      2907
           1       0.14      0.83      0.24        93

    accuracy                           0.83      3000
   macro avg       0.57      0.83      0.57      3000
weighted avg       0.97      0.83      0.89      3000



## Montando Rede Neural $\to$ 4 Camadas [Input, ReLU, Sigmoid, Output]

In [ ]:
"""
Necessary imports to create neural network
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential([
    # Input layer
    Dense(units= )

])